In [1]:
import pandas as pd

In [5]:
plm=pd.read_excel("C:/Users/claudio.pacheco/Downloads/plm_limpio.xlsx")

In [7]:
#Eliminar duplicados de nombre_producto
plm=plm.drop_duplicates(subset=['nombre_producto'], keep='first')
plm

,nombre_producto,sustancia,forma_farm,lab,presentacion
0,a.f. valdecasas,vitamina b9 (acido folico),tabletas,valdecasas,"1 frasco(s) , 20 tabletas , 1.0 miligramos"
10,a.s. cor,norfenefrina,solucion,sanfer,"1 frasco con gotero, 24 ml,"
11,abacavir,abacavir,tabletas,cryopharma,"1 caja , 60 tabletas , 300 miligramos"
12,abatoarin,tropisetron,capsulas,pisa,"1 caja , 1 capsulas , 5 miligramos"
15,abbatin,montelukast,tabletas masticables,corne,"1 caja,20 tabletas masticables,5 mg"
...,...,...,...,...,...
16634,zyrtec,cetirizina,solucion,armstrong,"1 caja, 1 frasco con gotero, 10 ml, 10 mg/ml"
16648,zytiga,abiraterona,tabletas,janssen-cilag,"1 caja, 1 frasco(s), 120 tabletas, 250 miligramos"
16650,zytokil,doxorubicina,solucion inyectable,pisa,NaN
16651,zyvoxam,linezolid,solucion inyectable,pfizer,"1 sobre con bolsa de infusion, 300 ml, 2 mg/ml"


In [8]:
import pandas as pd
import tabula
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
import time
import random

"""Esta función busca en la página de la COFEPRIS el registro sanitario de un medicamento
    
    """

def registro(medicamento):
    #función para cambiar la denominación del medicamento
    def denominacion(option_text):
        combo = driver.find_element(By.CLASS_NAME, "form-control")
        combo_select = Select(combo)
        combo_select.select_by_visible_text(option_text)
        cuadro_busqueda = driver.find_element(By.ID, "MainContent_txtSearchValue")
        cuadro_busqueda.send_keys(medicamento)
        button = driver.find_element(By.XPATH, "//button[contains(@onclick, '__doPostBack') and @class='btn btn-primary']")
        button.click()
    #Función para extraer los elementos de la tabla
    def elementos_tabla(driver):
        table_element = driver.find_element(By.CLASS_NAME, "gridView_basic")
        table_contents = table_element.get_attribute('outerHTML')
        return table_contents
    
    #Código para extraer los datos de la página
    url = "https://tramiteselectronicos02.cofepris.gob.mx/BuscadorPublicoRegistrosSanitarios/BusquedaRegistroSanitario.aspx"
    options = webdriver.ChromeOptions()
    options.add_argument('--incognito')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--disable-cache')
    options.add_argument('--disable-cookies')
    options.add_argument('--headless')
    driver = webdriver.Chrome(service=Service("C:/chromedriver.exe"), options=options)
    driver.get(url)
    #Denominación Distintiva
    denominacion("Denominación Distintiva")

    try:
        table_contents = elementos_tabla(driver)
    except NoSuchElementException:
        #Denominación Genérica
        denominacion("Denominación Genérica")
        try:
            table_contents = elementos_tabla(driver)
        except NoSuchElementException:
            driver.quit()
            return pd.DataFrame()

    soup = BeautifulSoup(table_contents, 'html.parser')
    registro=[]
    marca = []
    generica = []
    tipo_med=[]
    indica = []
    lab = []
    fabricante = []
    principio=[]

    for span in soup.find_all('span'):
        if span.has_attr('id'):
            if span['id'].startswith('MainContent_GV_RegistroSanitario_lblNumeroRegistro'):
                registro.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblDenominacionDistintiva'):
                marca.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblDenominacionGenerica'):
                generica.append(span.text)
            elif span["id"].startswith('MainContent_GV_RegistroSanitario_lblTipoMedicamento'):
                tipo_med.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblTitularRegistro'):
                lab.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblFabricanteMedicamento'):
                fabricante.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblPrincipioActivo'):
                principio.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblIndicacionTerapeutica'):
                indica.append(span.text)

    df = pd.DataFrame({'reg_san':registro,'marca': marca, 'den_gen': generica, 'tipo_med':tipo_med, 'titular': lab, 'fabricante': fabricante, 'principio': principio,'indicacion':indica,"busqueda":medicamento})
    df = df.apply(lambda x: x.astype(str).str.lower())
    df = df.apply(lambda x: x.astype(str).str.replace('\n', ''))
    driver.quit()
    return df

In [19]:
tabla=pd.DataFrame()
for i in plm["nombre_producto"]:
    #Incorporar búsqueda en una nueva variable
    busqueda=i
    #Ejecutar la función
    df=registro(busqueda)
    #Agregar el resultado a la tabla
    tabla=pd.concat([tabla,df],axis=0)
    time.sleep(random.randint(1,2)
tabla


,reg_san,marca,den_gen,tipo_med,titular,fabricante,principio,indicacion,busqueda
0,152m2015 ssa,colmesdant,colistimetato,de referencia,"especificos stendhal, s.a. de c.v.",patheon manufacturing services llc.,a07aa10 colistina.,antibiótico para infecciones ocasionadas por b...,colmesdant
